# Serving Models with Seldon Core - Sklearn Server

In this example, we'll use SeldonCore to easily deploy a pickled sklearn model as an endpoint with the ability to scale horizontally

We'll start by building a simple logistic regression on the Iris dataset

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from joblib import dump

In [3]:
X,y = load_iris(return_X_y=True)

In [9]:
lr = LogisticRegression(multi_class='ovr')

In [10]:
lr.fit(X,y)

/home/john/anaconda3/envs/kubeflow/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
# New Data
data = [[5,
         3,
         1.5,
         0.2]]


preds = lr.predict_proba(data)

In [ ]:
# Predictions for new data
for n in range(0,3):
    print(f"class{n}: ","{:.4%}".format(preds[0][n]))

## We'll save our model locally where Minikube can mount it

In [35]:
# Create a folder
!mkdir -p pv_storage/_storage/sklearn_iris

Pickled Joblib models *MUST* be named **model.joblib** for Seldon to pick them up correctly. 

If you would like to identify your models, you can do so by naming the directory

In [36]:
dump(lr,"./pv_storage/sklearn_iris/model.joblib")

['./seldon_models/sklearn_iris/model.joblib']

At this point, we'll need to mount the pv_storage file to minikube.

In a **new terminal session** make sure you're in the repository's root and run:

```
minikube mount $(pwd)/pv_storage/:/mkdata
```

This will allow minikube to access files in the pv_storage folder - MiniKube will see it as 'mkdata' 

This terminal session has to stay running as long as we're working with this model. A Persistent Volume + and Persistent Volume Claim would be a more long-term solution; or hosting the model in cloud storage


In [2]:
!cat sklearn_iris.yaml

apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: <<FILL IN>>
      name: classifier
    name: default
    replicas: 1
  volumes:
    name: host-mount
    hostpath:
      path: <<FILL IN>>

An simple yaml file that controls our model's deployment:

* Seldon Deployment
* Named sklearn
* Model is named iris (goes into pod names, endpoint names, etc)
* Its an sklearn model so we'll use an sklearn server (XGBoost and Tensorflow are also options)
    * Other models can be served with Seldon, but they require packaging the model into a container instead of pulling from a URI
* modelUri defines the path to the model.joblib file
    * This can be hosted many places:
        * A volume or PVC
        * Google Cloud Storage
        * AWS S3
        * Azure Blob

* Volumes tells k8s to make the volume we'll mount available to this pod


Fill in the location of your model (or use the sklean_iris_example.yaml file which has been completed to match the above host-mount configuration)

We can submit the process to our cluster using Kubectl:

In [ ]:
!kubectl apply -f sklearn_iris_example.yaml